In [ ]:
# !pip install simplejson

In [2]:
from pymongo import MongoClient
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import simplejson as json
import itertools
from functools import cmp_to_key
import networkx as nx

from IPython.display import display, Image, JSON
from ipywidgets import widgets, Image, HBox, VBox, Button, ButtonStyle

from lib.image_dedup import make_hashes, calculate_distance, hashes_diff

In [3]:
images_dir = Path('./images')
handmade_dir = Path('./handmade')
handmade_dir.mkdir(exist_ok=True)

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']
imagemeta = db['imagemeta']
imagededup = db['imagededup']

In [5]:
# imagededup.drop()
# imagededup.insert_many([i for i in imagemeta.find()])

# Load image metadata

In [6]:
imageDedup = [m for m in imagededup.find()]
imageDedup.sort(key=lambda x: x['image_id'])

In [7]:
phash_to_idx_mapping = {}
for i in range(len(imageDedup)):
    phash = imageDedup[i]['phash']
    l = phash_to_idx_mapping.get(phash, [])
    l.append(i)
    phash_to_idx_mapping[phash] = l
def phash_to_idx (phash):
    return phash_to_idx_mapping.get(phash, None)

In [8]:
image_id_to_idx_mapping = {imageDedup[i]['image_id']:i for i in range(len(imageDedup))}
def image_id_to_idx (image_id):
    return image_id_to_idx_mapping.get(image_id, None)

# Calculate distance

## Hash distance

In [9]:
image_hashes = [make_hashes(m) for m in imageDedup]

In [10]:
distance = calculate_distance(image_hashes)

In [11]:
# distance2 = np.ndarray([len(image_hashes), len(image_hashes)])
# for i in tqdm(range(len(image_hashes))):
#     for j in range(i+1):
#         diff = hashes_diff(image_hashes[i], image_hashes[j])
#         distance2[i, j] = diff
#         distance2[j, i] = diff
# np.array_equal(distance, distance2)

In [12]:
pdistance = calculate_distance(image_hashes, hash_type='phash')

## Find duplicated pairs from distance matrix

In [13]:
def set_distance (hashes, value, mat=distance):
    phash_x = hashes[0]
    phash_y = phash_x if len(hashes) == 1 else hashes[1]
    idx_x = phash_to_idx(phash_x)
    idx_y = phash_to_idx(phash_y)
    if idx_x == None or idx_y == None:
        return
    for s in itertools.product(idx_x, idx_y):
        i, j = s
        mat[i, j] = value
        mat[j, i] = value

def set_distance_pairs (phash_pairs, value, mat=distance):
    for p in phash_pairs:
        set_distance(list(p), value, mat=mat)

In [14]:
class PersistentSet (set):
    @staticmethod
    def load_set (file):
        if Path(file).exists():
            s = PersistentSet([frozenset(a) if isinstance(a, list) else a for a in json.load(open(file))])
        else:
            s = PersistentSet()
        s.set_file(file)
        return s

    def set_file (self, file):
        self.file = file

    def save (self):
        json.dump(list(self), open(self.file, 'w'), iterable_as_array=True)

    def persist_add (self, item):
        self.add(item)
        self.save()

    def persist_remove (self, item):
        self.remove(item)
        self.save()

In [15]:
auto_duplicated_image_phash_pairs = PersistentSet()
auto_duplicated_image_phash_pairs.set_file(handmade_dir/'duplicated_image_phash_pairs_auto.json')

In [16]:
for i in tqdm(range(distance.shape[0])):
    for j in range(i):
        if distance[i, j] <= 1: # checked, all distance <= 1 are duplicated
            auto_duplicated_image_phash_pairs.add(frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']]))

In [17]:
for i in tqdm(range(pdistance.shape[0])):
    for j in range(i):
        if pdistance[i, j] <= 1: # checked, all distance <= 1 are duplicated
            auto_duplicated_image_phash_pairs.add(frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']]))

In [18]:
auto_duplicated_image_phash_pairs.save()

## Apply information from meta data

In [19]:
duplicated_post_image_phash_pairs = PersistentSet()
duplicated_post_image_phash_pairs.set_file(handmade_dir/'duplicated_post_image_phash_pairs.json')

for p in posts.find():
    if 'duplicated_posts' not in p or len(p['duplicated_posts']) == 0:
        continue

    dp_phashes = [i['phash']
                    for dp in p['duplicated_posts']
                    for i in imagemeta.find({'post_id': dp})]
    if len(dp_phashes) > 1:
        print(f"More than 1 image {p['post_id']} {dp_phashes}")

    phashes = [i['phash'] for i in imagemeta.find({'post_id': p['post_id']})]
    for s in itertools.product(dp_phashes, phashes):
        fs = frozenset(s)
        if len(fs) > 1:
            duplicated_post_image_phash_pairs.add(fs)

duplicated_post_image_phash_pairs.save()

In [20]:
related_album_image_phash_pairs = PersistentSet()
related_album_image_phash_pairs.set_file(handmade_dir/'related_album_image_phash_pairs.json')

for album in {i['album'] for i in imagemeta.find({'album': {'$exists': True, '$ne': ''}})}:
    ra_phashes = [i['phash'] for i in imagemeta.find({'album': album})]
    if len(ra_phashes) <= 1:
        print(f"Only 1 or less image {album} {p['post_id']} {ra_phashes}")

    for s in itertools.product(ra_phashes, ra_phashes):
        fs = frozenset(s)
        if len(fs) > 1:
            related_album_image_phash_pairs.add(fs)

related_album_image_phash_pairs.save()

## Apply manual labeled data

In [21]:
duplicated_image_phash_pairs = PersistentSet.load_set(handmade_dir/'duplicated_image_phash_pairs.json')
not_duplicated_image_phash_pairs = PersistentSet.load_set(handmade_dir/'not_duplicated_image_phash_pairs.json')
related_image_phash_pairs = PersistentSet.load_set(handmade_dir/'related_image_phash_pairs.json')
invalid_image_phashes = PersistentSet.load_set(handmade_dir/'invalid_image_phashes.json')

In [22]:
set_distance_pairs(auto_duplicated_image_phash_pairs, 0)
set_distance_pairs(duplicated_post_image_phash_pairs, 0)
set_distance_pairs(duplicated_image_phash_pairs, 0)
set_distance_pairs(not_duplicated_image_phash_pairs, 60)
set_distance_pairs(related_album_image_phash_pairs, 60)
set_distance_pairs(related_image_phash_pairs, 60)

related_distance = np.full(distance.shape, 60)
set_distance_pairs(related_album_image_phash_pairs, 0, mat=related_distance)
set_distance_pairs(related_image_phash_pairs, 0, mat=related_distance)

# Human in the Loop

In [23]:
def make_dedup_box (idx_x, idx_y, default=None):
    image_x = imageDedup[idx_x]
    phash_x = image_x['phash']
    image_y = imageDedup[idx_y]
    phash_y = image_y['phash']
    hash_pair = frozenset([phash_x, phash_y])

    yes_btn = widgets.Button(description="Duplicated", button_style='success')
    no_btn = widgets.Button(description="Not", button_style='info')
    related_btn = widgets.Button(description="Related", button_style='warning')
    invalid_x_btn = widgets.Button(description="X Invalid")
    invalid_y_btn = widgets.Button(description="Y Invalid")
    reset_btn = widgets.Button(description="Reset")
    output = widgets.Output()

    def on_yes (btn):
        with output:
            if default == 'no' and hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            duplicated_image_phash_pairs.persist_add(hash_pair)
            print('Duplicated')

    def on_no (btn):
        not_duplicated_image_phash_pairs.persist_add(hash_pair)
        with output:
            print('Not')

    def on_related (btn):
        with output:
            if default == 'no' and hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            related_image_phash_pairs.persist_add(hash_pair)
            print('Related')

    def on_invalid_x (btn):
        invalid_image_phashes.persist_add(phash_x)
        with output:
            print('Invalid X')

    def on_invalid_y (btn):
        invalid_image_phashes.persist_add(phash_y)
        with output:
            print('Invalid Y')

    def on_reset (btn):
        with output:
            if hash_pair in duplicated_image_phash_pairs:
                duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Duplicated')
            if hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            if hash_pair in related_image_phash_pairs:
                related_image_phash_pairs.persist_remove(hash_pair)
                print('-Related')
            if phash_x in invalid_image_phashes:
                invalid_image_phashes.persist_remove(phash_x)
                print('-Invalid X')
            if phash_y in invalid_image_phashes:
                invalid_image_phashes.persist_remove(phash_y)
                print('-Invalid Y')
            print('Reset')

    yes_btn.on_click(on_yes)
    no_btn.on_click(on_no)
    related_btn.on_click(on_related)
    invalid_x_btn.on_click(on_invalid_x)
    invalid_y_btn.on_click(on_invalid_y)
    reset_btn.on_click(on_reset)

    if default == 'no':
        on_no(None)

    return HBox([VBox([yes_btn, no_btn, related_btn, invalid_x_btn, invalid_y_btn, reset_btn, output]),
                 widgets.Image(value=open(image_x['file_path'], 'rb').read(), width=250, height=150),
                 widgets.Image(value=open(image_y['file_path'], 'rb').read(), width=250, height=150)])

In [24]:
def potential_duplicates (threshold):
    for i in range(distance.shape[0]):
        for j in range(i):
            if distance[i, j] <= threshold:
                phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
                if (phash_pair not in auto_duplicated_image_phash_pairs and
                    phash_pair not in duplicated_post_image_phash_pairs and
                    phash_pair not in duplicated_image_phash_pairs and
                    phash_pair not in not_duplicated_image_phash_pairs and
                    phash_pair not in related_album_image_phash_pairs and
                    phash_pair not in related_image_phash_pairs):
                    yield (i, j)

In [25]:
distance_threshold = 12

In [26]:
# pdup = potential_duplicates(distance_threshold)

In [27]:
# for i in range(10):
#     try:
#         next_pdup = next(pdup)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
#     display(make_dedup_box(idx_x, idx_y, default=None if distance[idx_x, idx_y] < 9 else 'no'))

## Images with high variety

In [28]:
# interested_phashes = set()

In [29]:
# def potential_duplicates_high (threshold):
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if distance[i, j] >= threshold:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (phash_pair in duplicated_image_phash_pairs):
#                     interested_phashes.add(imageMetas[i]['phash'])
#                     interested_phashes.add(imageMetas[j]['phash'])
#                     yield (i, j)

In [30]:
# pduph = potential_duplicates_high(10)

In [31]:
# for i in range(100):
#     try:
#         next_pdup = next(pduph)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
# #     display(make_dedup_box(idx_x, idx_y))

In [32]:
# def potential_duplicates_interested (threshold):
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if distance[i, j] <= threshold:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (imageMetas[i]['phash'] in interested_phashes or
#                     imageMetas[j]['phash'] in interested_phashes) and (
#                     phash_pair not in auto_duplicated_image_phash_pairs and
#                     phash_pair not in duplicated_post_image_phash_pairs and
#                     phash_pair not in duplicated_image_phash_pairs and
#                     phash_pair not in not_duplicated_image_phash_pairs and
#                     phash_pair not in related_album_image_phash_pairs and
#                     phash_pair not in related_image_phash_pairs):
#                     yield (i, j)

In [33]:
# pdupi = potential_duplicates_interested(18)

In [34]:
# for i in range(10):
#     try:
#         next_pdup = next(pdupi)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
#     display(make_dedup_box(idx_x, idx_y))

In [35]:
# def potential_duplicates_related ():
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if related_distance[i, j] == 0:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (phash_pair not in related_album_image_phash_pairs):
#                     yield (i, j)

In [36]:
# pdupr = potential_duplicates_related()

In [37]:
# for i in range(10):
#     try:
#         next_pdup = next(pdupr)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {pdistance[idx_x, idx_y]} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
#     display(make_dedup_box(idx_x, idx_y))

# Consolidate

## Related images

In [38]:
related_images = [[imageDedup[idx]['image_id'] for idx in c]
                     for c in nx.components.connected_components(nx.Graph(related_distance <= 1))
                     if len(c) > 1]
len(related_images)

72

In [39]:
for ids in related_images:
    for i in ids:
        imageMeta = imagemeta.find_one({'image_id': i})
        ri = [r for r in set(imageMeta.get('related_images', []) + ids) if r != i]
        imagededup.update_one({'image_id': i}, {'$set': {'related_images': ri}})

## Duplicated images

In [43]:
class ImageDedup ():
    _attrs = [
        'id',
        'post_id',
        'datetime',
        'url',
        'title',
        'content',
        'author',
        'removed',
        'ups',
        'num_comments',
        'external_link',
        'source',
        'source_platform',
        'source_url',
        'tags',
        'labels',
        'media_type',

        'thumbnail',
        'preview',
        'external_link',
        'archive',

        'image_id',
        'short_image_id',
        'album',
        'index_in_album',
        'file_path',
        'ext',
        'animated',
        'size',
        'width',
        'height',
        'ahash',
        'phash',
        'pshash',
        'dhash',
        'whash',

        'duplicated_posts',
        'related_images',
        'duplicated_images'
    ]

    rank = {
        'dataisugly': 3,
        'wtf-viz': 2,
        'badvisualisations': 1
    }

    def __init__ (self, imageMetas=[]):
        if len(imageMetas) == 0:
            raise Exception('Empty imageFiles array.')
        self._imageMetas = imageMetas
        self._image_ids = [i['image_id'] for i in imageMetas]
        self._image_order = self.sort_images()
        self._post_ids = {i['post_id'] for i in imageMetas}
        self._posts = [posts.find_one({'post_id': i}) for i in self._post_ids]
        self._post_order = self.sort_posts()
        for k, v in self.main_image.items():
            if k in ['duplicated_posts', 'related_images']:
                continue
            setattr(self, k, v)
        for k, v in self.main_post.items():
            if k in ['duplicated_posts', 'related_images']:
                continue
            if k in ['preview', 'thumbnail']:
                setattr(self, f"{k}_url", v)
            else:
                setattr(self, k, v)

    def digest (self):
        return {a:getattr(self, a) for a in ImageDedup._attrs if hasattr(self, a)}

    @property
    def duplicated_posts (self):
        post_ids = list(self._post_ids)
        for p in self._posts:
            if 'duplicated_posts' in post_ids:
                post_ids += p['duplicated_posts']
        return [i for i in set(post_ids) if i != self.post_id]

    @property
    def duplicated_images (self):
#         return self._image_ids
        return [i for i in self._image_ids if i != self.image_id]

    @property
    def related_images (self):
        return [ri for i in self._imageMetas for ri in i.get('related_images', []) if ri != self.image_id]

    @property
    def main_post (self):
        return self._post_order[0]

    def post_score (self, post):
        score = post['ups'] if 'ups' in post else 0
        score += 5 * post['num_comments'] if 'num_comments' in post else 0
        return score

    def source_rank (self, source):
        return ImageDedup.rank.get(source, 0)

    def sort_posts (self):
        def preferred (post_x, post_y):
            rank_x = self.source_rank(post_x['source'])
            rank_y = self.source_rank(post_y['source'])
            if rank_x != rank_y:
                return rank_y - rank_x
            post_score_x = self.post_score(post_x)
            post_score_y = self.post_score(post_y)
            return post_score_y - post_score_x

        return sorted(self._posts, key=cmp_to_key(preferred))

    @property
    def main_image (self):
        return self._image_order[0]

    def sort_images (self):
        def preferred (image_x, image_y):
            if image_x['animated'] != image_y['animated']:
                if image_x['animated']:
                    return -1
                if image_y['animated']:
                    return 1

            pixels_x = image_x['width'] * image_x['height']
            pixels_y = image_y['width'] * image_y['height']

            if pixels_x == pixels_y:

                if image_x['ext'] != image_y['ext']:
                    if image_x['ext'] == '.png':
                        return -1
                    if image_y['ext'] == '.png':
                        return 1

                if image_x['size'] != image_y['size']:
                    return image_y['size'] - image_x['size']

                return self.source_rank(image_y['source']) - self.source_rank(image_x['source'])
            else:
                return pixels_y - pixels_x

        return sorted(self._imageMetas, key=cmp_to_key(preferred))

In [41]:
duplicated_images = [[imageDedup[idx]['image_id'] for idx in c]
                     for c in nx.components.connected_components(nx.Graph(distance <= 1))
                     if len(c) > 1]
len(duplicated_images)

744

In [44]:
for ids in duplicated_images:
    imagedd = ImageDedup([imagemeta.find_one({'image_id': i}) for i in ids])
    for i in imagedd.duplicated_images:
        imagededup.delete_one({'image_id': i})
    imagededup.replace_one({'image_id': imagedd.image_id}, imagedd.digest(), upsert=True)

In [ ]:
# duplicated_image_ids = [c
#                      for c in nx.components.connected_components(nx.Graph(distance <= 1))
#                      if len(c) > 1]
# len(duplicated_image_ids)
# for idxs in duplicated_image_ids:
#     print(f"{[imageDedup[i]['image_id'] for i in idxs]}")
#     if len(idxs) >= 4:
#         display(HBox([
#             widgets.Image(value=open(imageDedup[i]['file_path'], 'rb').read(), width=100, height=100)
#             for i in idxs]))